In [2]:
!pip install tensorflow
!pip install keras


  Using cached tensorflow-2.17.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-1-py2.py3-none-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.4.0-cp312-cp312-macosx_10_9_universal2.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.6 kB)
  Using cached w

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.initializers import HeNormal,random_uniform
import time
from keras.layers import Add, AveragePooling2D, MaxPooling2D, Flatten, Dense, Activation, Input, BatchNormalization, MaxPooling2D, Conv2D, ZeroPadding2D


In [4]:
def identity_block(X, f, filters, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters 
    X_shortcut = X
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [5]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X

    X = Conv2D(F1, (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(F2,  kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    X_shortcut = Conv2D(F3, (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = HeNormal(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X) 
    
    return X

In [6]:
def ResNet30(input_shape = (64, 64, 3), classes = 2):
    """
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    """
    X_input = Input(input_shape)


    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = HeNormal(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')

    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')

    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')

    X = AveragePooling2D((2, 2), name='avg_pool')(X)

    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = HeNormal(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name='ResNet30')

    return model

In [8]:
from keras.models import Model, load_model

In [9]:
model = ResNet30(input_shape = (64, 64, 3), classes = 2)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    r'/Users/yeshnavya/Documents/archive/test',
    target_size=(64, 64),  # Match input shape
    batch_size=32,
    class_mode='categorical'  # Since classes=2
)

test_generator = datagen.flow_from_directory(
    r'/Users/yeshnavya/Documents/archive/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

Found 20000 images belonging to 2 classes.
Found 100000 images belonging to 2 classes.


In [14]:
import tensorflow as tf
tf.keras.backend.clear_session()


model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=10  # Adjust based on your requirements
)

Epoch 1/10


/Users/yeshnavya/Documents/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


625/625 ━━━━━━━━━━━━━━━━━━━━ 538s 850ms/step - accuracy: 0.7634 - loss: 0.6389 - val_accuracy: 0.5207 - val_loss: 2.6706
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 172us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


2024-08-17 22:48:03.513325: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-08-17 22:48:03.560703: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


625/625 ━━━━━━━━━━━━━━━━━━━━ 550s 881ms/step - accuracy: 0.8608 - loss: 0.3381 - val_accuracy: 0.7525 - val_loss: 0.6698
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 82us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10


2024-08-17 22:57:13.933541: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


625/625 ━━━━━━━━━━━━━━━━━━━━ 556s 890ms/step - accuracy: 0.8723 - loss: 0.3118 - val_accuracy: 0.8863 - val_loss: 0.2735
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 88us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 585s 937ms/step - accuracy: 0.9179 - loss: 0.2080 - val_accuracy: 0.8912 - val_loss: 0.2869
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 104us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10


2024-08-17 23:16:15.707741: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


625/625 ━━━━━━━━━━━━━━━━━━━━ 586s 938ms/step - accuracy: 0.9133 - loss: 0.2133 - val_accuracy: 0.8366 - val_loss: 0.4458
Epoch 10/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 66us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [15]:
model.save('/Users/yeshnavya/Documents/archive/resnet_30_shallow.h5')

In [17]:
model.save_weights('/Users/yeshnavya/Documents/archive/resnet_30_shallow_weights.weights.h5')

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Create a checkpoint callback
checkpoint = ModelCheckpoint('path_to_your_model/weights.{epoch:02d}-{val_loss:.2f}.h5',
                             save_weights_only=False,  # Set to True if you only want to save weights
                             save_best_only=True,      # Saves only the best model according to the monitored metric
                             monitor='val_loss',       # Metric to monitor
                             mode='min')               # Save when the monitored metric decreases

# Pass the callback to model.fit
model.fit(x_train, y_train, 
          validation_data=(x_val, y_val),
          epochs=10, 
          callbacks=[checkpoint])


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=path_to_your_model/weights.{epoch:02d}-{val_loss:.2f}.h5